In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import sqlite3
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [2]:

# Path to SQLite database
db_path = 'database/stocks_data.db'

# Load data from SQLite
with sqlite3.connect(db_path) as conn:
    query = "SELECT * FROM processed_stocks"
    data = pd.read_sql(query, conn)
print(f"Loaded processed data: {data.shape[0]} rows")


Loaded processed data: 393722 rows


In [3]:

# Step 1: Set default ticker
default_ticker = 'XOM'

# Step 2: Filter data for the default ticker
ticker_data = data[data['Ticker'] == default_ticker]
print(f"Loaded data for {default_ticker}: {ticker_data.shape[0]} rows")


Loaded data for XOM: 68979 rows


In [4]:

# Define features and target
features = ['7-day MA', '14-day MA', 'Volatility', 'Lag_1', 'Lag_2']
target = 'Adj Close'

X = ticker_data[features]
y = ticker_data[target]


In [5]:

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [6]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape input for LSTM (samples, timesteps, features)
X_train_scaled = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_scaled = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


In [7]:

# Build the optimized LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1, X_train_scaled.shape[2])))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))  # Fully connected output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Callbacks for better training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=100,  # Start with 100 epochs
    batch_size=128,  # Reduced batch size
    callbacks=[early_stopping]
)


Epoch 1/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1407.4867 - val_loss: 9.1065
Epoch 2/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 65.4836 - val_loss: 5.0618
Epoch 3/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 62.0061 - val_loss: 5.2042
Epoch 4/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 59.4145 - val_loss: 3.1315
Epoch 5/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 58.0178 - val_loss: 3.0330
Epoch 6/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 57.2242 - val_loss: 2.1004
Epoch 7/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 53.7329 - val_loss: 1.9222
Epoch 8/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 50.8528 - val_loss: 3.1664
Epoch 9/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 49.0603 - val_loss: 3.1999
Epoch 10/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 47.9251 - val_loss: 1.7840
Epoch 11/50
863/863 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 45.8780 - val_loss: 2.9950
Epoch 12/50
863/863 ━━━━━━━━━━━━━━━━━━━

In [9]:

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")


432/432 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step
Model Evaluation:
Mean Squared Error (MSE): 0.93
Mean Absolute Error (MAE): 0.65
R-squared (R²): 1.00


In [10]:

# Save the trained model and scaler
model_filename = f'models/model_{default_ticker}_lstm.h5'
scaler_filename = f'models/scaler_{default_ticker}_lstm.pkl'

model.save(model_filename)
joblib.dump(scaler, scaler_filename)

print(f"{default_ticker} model saved as '{model_filename}'")
print(f"{default_ticker} scaler saved as '{scaler_filename}'")

XOM model saved as 'models/model_XOM_lstm.h5'
XOM scaler saved as 'models/scaler_XOM_lstm.pkl'
